In [3]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten, Input, ZeroPadding2D
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.python.client import device_lib
from keras.callbacks import EarlyStopping
print(device_lib.list_local_devices())
%matplotlib inline

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10417285223965265910
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3135687884
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11832656507221156993
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [5]:
## Loading test and train data
## Converting to 3D array
## Loading dataset 
dogs = []
cats = []
for i in range(1,4001):
    img_dog = image.load_img(r"D:\Columbia_University\MS AI\Lecture 2\Cats and Dogs\dataset\training_set\dogs\dog." + str(i) 
                         + ".jpg", target_size=(128,128))
    img_dog = image.img_to_array(img_dog)
    dogs.append(img_dog)
    
    ## For cat
    img_cat = image.load_img(r"D:\Columbia_University\MS AI\Lecture 2\Cats and Dogs\dataset\training_set\cats\cat." + str(i) 
                         + ".jpg", target_size=(128,128))
    img_cat = image.img_to_array(img_cat)
    cats.append(img_cat)

## Converting to array
dogs = np.array(dogs)
cats = np.array(cats)


x_train = np.vstack((dogs, cats))/255
# x_train = x_train.reshape(x_train.shape[0], 224, 224, 3)/255
y_train = np.concatenate((np.ones((4000,)), np.zeros((4000,))))

x_train.shape

s = np.arange(x_train.shape[0])
np.random.shuffle(s)

## Shuffling
x_train = x_train[s]
y_train = y_train[s]

## Loading test dataset 
dogs = []
cats = []
for i in range(4001,5001):
    img_dog = image.load_img(r"D:\Columbia_University\MS AI\Lecture 2\Cats and Dogs\dataset\test_set\dogs\dog." + str(i) 
                         + ".jpg", target_size=(128,128))
    img_dog = image.img_to_array(img_dog)
    dogs.append(img_dog)
    
    ## For cat
    img_cat = image.load_img(r"D:\Columbia_University\MS AI\Lecture 2\Cats and Dogs\dataset\test_set\cats\cat." + str(i) 
                         + ".jpg", target_size=(128,128))
    img_cat = image.img_to_array(img_cat)
    cats.append(img_cat)

## Converting to array
dogs = np.array(dogs)
cats = np.array(cats)

## Converting to 3D array
x_test = np.vstack((dogs, cats))/255
# x_train = x_train.reshape(x_train.shape[0], 224, 224, 3)/255
y_test = np.concatenate((np.ones((1000,)), np.zeros((1000,))))

print(x_test.shape)

s = np.arange(x_test.shape[0])
np.random.shuffle(s)

## Shuffling
x_test = x_test[s]
y_test = y_test[s]

x_test.shape

(2000, 128, 128, 3)


(2000, 128, 128, 3)

In [ ]:
## Initial Try

"""## Defining Architechture
model = Sequential()

### 1st Layer
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', input_shape = (128, 128, 3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2),))

### 2nd Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),))

# ### 3rd Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2),))

# ### 4th Layer
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2),))

### Flatten
model.add(Flatten())

### 1st Layer
model.add(Dense(128, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
# model.add(Dropout(0.4))

### 2nd Layer
model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

### Output
model.add(Dense(1))
model.add(Activation('sigmoid'))"""

In [9]:
def model(input_shape):
    ## Input
    X_input = Input(input_shape)
    
    ## Zero Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    ## 1st Layer
    X = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    
    ## 2nd Layer
    X = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    
    ## 3rd Layer
    X = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    
    ## Flatten
    X = Flatten()(X)
    
    ## 1st Dense Layer
    X = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(X)
#     X = Dropout(0.5)(X)
    
    ## 2nd Layer
    X = Dense(64, activation='relu')(X)
    X = Dropout(0.5)(X)
    
    ## Output Layer
    X = Dense(1, activation='sigmoid')(X)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Cats&Dogs')
    
    return model

In [10]:
## Compiling model
model = model((128, 128, 3))
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'],)
model.summary()

Model: "Cats&Dogs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 134, 134, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 134, 134, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 67, 67, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 67, 67, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 33, 33, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 33, 33, 64)        36

In [11]:
# ## Training the model
training_log = model.fit(x=x_train, y=y_train, validation_split=0.05, batch_size=32, epochs=15,)

Train on 7600 samples, validate on 400 samples
Epoch 1/15
7600/7600 [==============================] - 18s 2ms/step - loss: 0.7826 - accuracy: 0.5675 - val_loss: 0.6635 - val_accuracy: 0.6350
Epoch 2/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.6590 - accuracy: 0.6343 - val_loss: 0.6466 - val_accuracy: 0.6750
Epoch 3/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.6128 - accuracy: 0.6909 - val_loss: 0.6027 - val_accuracy: 0.7025
Epoch 4/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.5856 - accuracy: 0.7113 - val_loss: 0.4939 - val_accuracy: 0.7675
Epoch 5/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.5570 - accuracy: 0.7430 - val_loss: 0.5063 - val_accuracy: 0.7925
Epoch 6/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.5503 - accuracy: 0.7437 - val_loss: 0.4938 - val_accuracy: 0.7700
Epoch 7/15
7600/7600 [==============================] - 17s 2ms/step - loss: 0.5201

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)

2000/2000 [==============================] - 2s 844us/step


In [14]:
print(f"The accuracy is {accuracy}")

The accuracy is 0.7885000109672546
